In [1]:
import warnings
warnings.simplefilter("ignore")

## Escalamiento de los datos

In [2]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

<div class="alert alert-success">
  Diversos algoritmos son sensibles a la escala en la que viene cada feature. **Re-escalarlos** puede traer significativas mejoras de rendimiento.
</div>

Existen distintas estrategias de escalamiento de tus features, pero **la más común es la estandarización** donde convertimos la variable para que la distribución de esta siga una distribución que es Gaussiana de media 0 y de desviación estandar 1.

In [3]:
from sklearn.model_selection import train_test_split

X = pd.read_csv('intermediate_results/X.csv')
print(X.shape)
y = X['worldwide_gross']
X = X.drop('worldwide_gross',axis=1)
X_train, X_test, y_train, y_test = train_test_split(X,y)

(2221, 10)


In [4]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [5]:
scaler.mean_

array([5.24557442e+07, 2.00619459e+03, 6.69896053e+04, 1.10868468e+02,
       1.37154036e+04, 5.34982804e+07, 6.33519520e+00, 1.79844359e+07,
       2.27848048e+03])

In [6]:
scaler.scale_

array([4.63224787e+07, 5.49840006e+00, 9.19082652e+05, 1.99564721e+01,
       2.33710051e+04, 6.98072833e+07, 1.01822361e+00, 2.17975695e+07,
       1.19034942e+03])

In [7]:
X.values

array([[4.25000000e+08, 2.00900000e+03, 1.78000000e+00, ...,
        7.90000000e+00, 7.70254810e+07, 3.45200000e+03],
       [3.00000000e+08, 2.00700000e+03, 2.35000000e+00, ...,
        7.10000000e+00, 1.39802190e+08, 4.36200000e+03],
       [3.00000000e+08, 2.01500000e+03, 2.35000000e+00, ...,
        6.80000000e+00, 7.04031480e+07, 3.92900000e+03],
       ...,
       [1.05000000e+07, 2.00700000e+03, 2.35000000e+00, ...,
        6.50000000e+00, 4.71234100e+06, 2.15500000e+03],
       [1.01000000e+07, 2.01000000e+03, 3.68397729e+06, ...,
        5.30000000e+00, 6.42156000e+05, 8.20000000e+01],
       [1.01000000e+07, 2.00700000e+03, 1.85000000e+00, ...,
        5.80000000e+00, 6.35000000e+05, 8.57000000e+02]])

In [8]:
scaler.transform(X_train)

array([[-0.59270887, -0.21726222, -0.07288491, ..., -0.42740631,
         0.18235047,  0.61034139],
       [ 3.40103251,  0.51022213, -0.07288491, ..., -0.32919606,
         4.17394568,  1.64281133],
       [ 0.37874173,  1.23770648, -0.07288491, ...,  0.16185522,
         0.56838255,  0.68846971],
       ...,
       [-0.80858679,  0.32835105, -0.07288491, ..., -0.2309858 ,
        -0.82348823, -1.8284383 ],
       [ 2.64545981,  1.60144866, -0.07288545, ...,  1.92963981,
         3.3240328 ,  1.40086557],
       [ 1.88988712,  1.78331975, -0.07288491, ...,  0.849327  ,
         1.06881669,  1.40842637]])

In [9]:
X_train_scaled, X_test_scaled = (scaler.transform(X_train), scaler.transform(X_test))

In [10]:
from sklearn.linear_model import Lasso

model = Lasso()
model_scaled = Lasso()

model.fit(X_train,y_train)
model_scaled.fit(X_train_scaled,y_train)

Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

In [11]:
print(model.score(X_test,y_test))
print(model_scaled.score(X_test_scaled,y_test))

0.8277983924252991
0.8277983909722307


<div class="alert alert-success">
  Los modelos de regresión no se ven afectados por el escalamiento de las features. Los de clasificación sí. 
</div>

## Simplificar las transformaciones con pipelines

<div class="alert alert-info">
  Para hacer tu código más reproducible, y para evitar tener que aplicar multiples veces una misma transformación te recomendamos utilizar <code> sklearn.pipeline.make_pipeline </code> que permite encadenar transformaciones a tus modelos.
</div>

In [12]:
from sklearn.pipeline import make_pipeline

model_scaled = make_pipeline(StandardScaler(),
                            Lasso())

model_scaled.fit(X_train,y_train)

Pipeline(memory=None,
     steps=[('standardscaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('lasso', Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False))])

In [13]:
print(model_scaled.score(X_test,y_test))

0.8277983909722307


## Crear nuevas features de forma automática

In [14]:
A = np.arange(6).reshape(3, 2)
A

array([[0, 1],
       [2, 3],
       [4, 5]])

In [15]:
from sklearn.preprocessing import PolynomialFeatures

transformer = PolynomialFeatures(2)
transformer.fit_transform(A)

array([[ 1.,  0.,  1.,  0.,  0.,  1.],
       [ 1.,  2.,  3.,  4.,  6.,  9.],
       [ 1.,  4.,  5., 16., 20., 25.]])

``PolynomialFeatures`` transforma una matriz $(A1,A2)$ a $(1,A1,A2,A1^2,A1\cdot A2,A2^2)$

In [16]:
X.shape

(2221, 9)

In [17]:
transformer = PolynomialFeatures(2)
transformer.fit_transform(X).shape

(2221, 55)

In [18]:
model_poly = make_pipeline(PolynomialFeatures(2),
                          Lasso())
model_poly.fit(X_train,y_train)
model_poly.score(X_test,y_test)

0.811601812772682

In [19]:
model = Lasso()
model.fit(X_train,y_train)
model.score(X_test,y_test)

0.8277983924252991

## Crear features categóricas

<div class="alert alert-success">
  En terminos de Machine Learning a las features que pueden tomar un número finito de valores se les llama categóricas. Ejemplos para esto són: género, páis, grado académico, etc.
</div>

Un mapeo del tipo $\{Perú, Chile, Colombia, Venezuela\} \rightarrow \{1, 2, 3, 4\}$ tiene el problema de asignarle un ordén a los valores posibles de la categoría. Este orden impacta de distintas maneras los algoritmos de Machine Learning, por ejemplo aquellos que dependen de la topología de $R^n$ y de la función de distancia entre puntos en este espacio, considerarán que ciertas categorías se encuentran más cercanas unas de otras, siendo que esto es generado puramente artificialmente por el encoding, y no por las datos per se. 

Para no introducir información falsa o erronéa en nuestro modelos existen formas más inteligentes de encodear nuestros datos.

_**Encoding one-hot**_

Este encoding consiste en asignarle una columna a cada categoría y rellenarla con 0 y 1 de la forma siguiente:

In [20]:
d = pd.DataFrame([['Chile','Colombia','Colombia','Venezuela'],['hombre','mujer','hombre','mujer']])
d = d.T
d.columns = pd.Index(['pais','genero'])
d

,pais,genero
0,Chile,hombre
1,Colombia,mujer
2,Colombia,hombre
3,Venezuela,mujer


In [21]:
pd.get_dummies(d)

,pais_Chile,pais_Colombia,pais_Venezuela,genero_hombre,genero_mujer
0,1,0,0,1,0
1,0,1,0,0,1
2,0,1,0,1,0
3,0,0,1,0,1


Sklearn también ofrece un objeto OneHotEncoder pero es un poco más díficil de utilizar, así que por criterios pedagogicos hemos elegido ``pd.get_dummies``. Sin embargo el objeto de sklearn tiene la ventaja de ser pipeable, por lo que es bueno considerarlo para ciertos casos de uso.

Cuantas columnas generaríamos con un one-hot encoding de nuestras features categóricas?

In [22]:
movies_obj = pd.read_csv('intermediate_results/movies_obj.csv')

In [23]:
movies_obj.apply(pd.Series.nunique).sort_values()

color                2
content_rating      18
language            47
country             65
genres             914
actor_1_name      2097
director_name     2398
actor_2_name      3032
actor_3_name      3521
plot_keywords     4760
movie_title       4917
dtype: int64

Las features más informativas son las del casting. Si embargo haciendo un one-hot encoding de estas estaríamos aumentando la dimensión por 2000 y algo!!

## Encoding Binario

Esta técnica no es canónica por lo que tendremos que buscarla en otra librería. Sin embargo el autor tuvo la buena idea de hacer su API compatible con la de sklearn, así que no tendremos ninguna dificultad en usarla.

$$ Categoria \rightarrow Numero \rightarrow Binario \rightarrow Columnas $$

In [24]:
!pip install category_encoders

In [25]:
# categoricals = pd.read_csv('intermediate_results/categoricals.csv').set_index('Unnamed: 0')
categoricals = pd.read_csv('intermediate_results/categoricals.csv')

In [26]:
categoricals.head(2)

,director_name,actor_1_name
0,James Cameron,CCH Pounder
1,Gore Verbinski,Johnny Depp


In [27]:
# categoricals = categoricals.reset_index(drop=True).fillna(0)

In [28]:
X_binenc = pd.concat([X, categoricals], axis=1)

In [29]:
X_binenc.shape

(2221, 11)

In [30]:
import category_encoders as ce
encoder = ce.BinaryEncoder(cols=['actor_1_name','director_name'])

In [31]:
encoder.fit_transform(X_binenc).shape

(2221, 30)

In [32]:
X_binenc = encoder.fit_transform(X_binenc)
X_binenc.head()

,actor_1_name_0,actor_1_name_1,actor_1_name_2,actor_1_name_3,actor_1_name_4,actor_1_name_5,actor_1_name_6,actor_1_name_7,actor_1_name_8,actor_1_name_9,...,director_name_10,production_budget,title_year,aspect_ratio,duration.1,cast_total_facebook_likes,budget,imdb_score,opening_gross,screens
0,0,0,0,0,0,0,0,0,0,0,...,0,425000000.0,2009.0,1.78,178.0,4834.0,237000000.0,7.9,77025481.0,3452.0
1,0,0,0,0,0,0,0,0,0,1,...,1,300000000.0,2007.0,2.35,169.0,48350.0,300000000.0,7.1,139802190.0,4362.0
2,0,0,0,0,0,0,0,0,1,0,...,0,300000000.0,2015.0,2.35,148.0,11700.0,245000000.0,6.8,70403148.0,3929.0
3,0,0,0,0,0,0,0,0,1,1,...,1,275000000.0,2012.0,2.35,164.0,106759.0,250000000.0,8.5,160887295.0,4404.0
4,0,0,0,0,0,0,0,0,0,1,...,1,275000000.0,2013.0,2.35,150.0,45757.0,215000000.0,6.5,29210849.0,3904.0


In [33]:
Xb_train, Xb_test, y_train, y_test = train_test_split(X_binenc,y)

In [34]:
X_train, X_test = (Xb_train[X.columns],Xb_test[X.columns])

In [35]:
model_binenc = Lasso()
model = Lasso()

In [36]:
model_binenc.fit(Xb_train,y_train)
model.fit(X_train,y_train)

Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

In [37]:
print(model_binenc.score(Xb_test,y_test))
print(model.score(X_test,y_test))

0.8519782258046313
0.8544810517023788


Aumentamos el rendimiento de nuestro algoritmo pero no de forma significativa. Mantengamos entonces la dimensionalidad de nuestro espacio de features baja, y vamos a buscar modelos más complejos.

## Conocimiento experto

<div class="alert alert-success">
  Una grán parte del diseño de las features pasa por un **conocimiento espécifico del dominio en el que se esta trabajando**. <br>
  Por ejemplo para analizar una imagen nuestro cerebro no se concentra en los millones de pixeles de una imagen, pero sólo en algunos relevantes como los de los contornos. Durante un buen tiempo **los sistemas de visión de computadores encodeaban features que traducían este conocimiento experto (contornos).** <br>
  Una de las únicas formas de obtener este conocimiento de forma sistemática es ir a bucear en repositorios de papers de Machine Learning como Arxiv, y estudiar la investigación que se ha hecho sobre el dominio específico.
</div>

## Más datos de calidad

Nada le gana conseguir más datos que sean encodeables en features de calidad.

_**Píramide de Maslow del Machine Learning**_

<img src="img/maslow.png">

Contamos con la base de datos de ganancias de las péliculas el primer fin de semana de exhibición, así como la cantidad de cines en la que fue estrenada.

In [38]:
pd.read_csv('datasets/opening_df.csv').head() # Ya usado desde el notebook 9

,Unnamed: 0,movie_title,opening_gross,screens
0,0,10 Days in a Madhouse,2451.0,10.0
1,1,10 Things I Hate About You,8330681.0,2271.0
2,2,102 Dalmatians,19883351.0,2704.0
3,3,12 Rounds,5329240.0,2331.0
4,4,12 Years a Slave,923715.0,19.0


Puedes mejorar considerablemente nuestra predicción?